In [47]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers
solvers.options['show_progress'] = False
from scipy.stats import chi2

## Data Import

In [48]:
mat = scipy.io.loadmat('Data/hw6data.mat')

In [49]:
cov = mat['Q']
mu = mat['mu'].reshape(-1)
rtn_series = mat['R']

In [50]:
sample_mean = np.mean(rtn_series,axis = 1)
sample_cov = np.cov(rtn_series)

In [51]:
sigma = np.diag(np.diag(cov/len(rtn_series[0])))

In [67]:
#Parameter Setting
k_list = np.arange(0.05, 1.05, 0.05)
iter_num = 200
t_list = np.arange(10, 110, 10)

## Sensitivity Analysis

### Traditional Markovitz Optimization

In [53]:
def Markovitz_basic_portfolio(mean, cov, t):
    a = 2/t
    N = len(mean)
    
    P = matrix(a*cov)
    q = matrix((-1)*mean)
    G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
    h = matrix(np.array([0.0 for i in range(N)]), tc='d')
    A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
    b = matrix(np.array([1.0]), tc='d')
    sol = solvers.qp(P,q,G,h,A,b)
    w = sol['x']
    CE_0 = -sol['primal objective']
    return w, CE_0

In [54]:
def Markovitz_sensitivity_analysis(mean, cov, t, k, objective):
    N = len(mean)
    a = 2/t
    
    if objective == 'mean':
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
        P = matrix(a*cov)
        q = matrix((-1)*mean_new)
    elif objective == 'variance':
        flag = 0
        while flag == 0:
            z = np.random.normal(size = N)*k + 1
            var = np.diag(cov)
            var_new = np.multiply(var, z)
            cov_new = cov + np.diag(var_new - var)
            flag = np.all(np.linalg.eigvals(cov_new) >= 0)
        P = matrix(a*cov_new)
        q = matrix((-1)*mean)
    elif objective == 'covariance':
        flag = 0
        while flag == 0:
            z = np.random.normal(size = N**2)*k + 1
            var = np.diag(cov)
            upper = np.triu(cov)
            cov_new = np.multiply(upper.reshape(-1), z).reshape((N,N))
            cov_new += cov_new.T
            cov_new = cov_new - np.diag(np.diag(cov_new)) + np.diag(var)
            flag = np.all(np.linalg.eigvals(cov_new) >= 0)
        P = matrix(a*cov_new)
        q = matrix((-1)*mean)
    
    G = matrix(np.diag([-1.0 for i in range(N)]), tc='d')
    h = matrix(np.array([0.0 for i in range(N)]), tc='d')
    A = matrix(np.array([[1.0 for i in range(N)]]), tc='d')
    b = matrix(np.array([1.0]), tc='d')

    sol = solvers.qp(P,q,G,h,A,b)
    w_new = sol['x']
    CE_x = np.dot(mean,w_new)[0] - 0.5*a*np.dot(np.dot(w_new.T, cov), w_new)[0][0]
    return CE_x

In [45]:
w0, CE_0 = Markovitz_basic_portfolio(sample_mean, sample_cov, t)

In [46]:
obj_list = ['mean', 'variance', 'covariance']
for obj in obj_list:
    print('The effects of errors in '+obj+': ')
    for k in k_list:
        CEL = 0
        for i in range(iter_num):
            CE_x = Markovitz_sensitivity_analysis(sample_mean, sample_cov, t, k, obj)
            CEL += (1 - CE_x/CE_0)
        CEL /= iter_num
        print('When k = ',k, 'the average cash equivalent loss is: ', CEL)

The effects of errors in mean: 
When k =  0.05 the average cash equivalent loss is:  0.0007343719382011349
When k =  0.1 the average cash equivalent loss is:  0.015146030521431812
When k =  0.15000000000000002 the average cash equivalent loss is:  0.0435086285609816
When k =  0.2 the average cash equivalent loss is:  0.06951792920332017
The effects of errors in variance: 
When k =  0.05 the average cash equivalent loss is:  2.0844720038937225e-09
When k =  0.1 the average cash equivalent loss is:  5.769371621555664e-09
When k =  0.15000000000000002 the average cash equivalent loss is:  1.0272693648320323e-08
When k =  0.2 the average cash equivalent loss is:  1.471702436213936e-08
The effects of errors in covariance: 
When k =  0.05 the average cash equivalent loss is:  2.832152582824913e-09
When k =  0.1 the average cash equivalent loss is:  5.530207074870101e-09
When k =  0.15000000000000002 the average cash equivalent loss is:  8.321230741126229e-09
When k =  0.2 the average cash eq

### Robust Optimization 

In [40]:
def Robust_portfolio(mean, cov, sigma, t, eta):
    N = len(mean)
    a = 2/t
    kappa = np.sqrt(chi2.ppf(eta, df = N))
    L = np.linalg.cholesky(sigma)
    
    P_0 = np.hstack([a*cov, np.array([[0.0] for i in range(N)])])
    P_0 = np.vstack([P_0, np.array([0.0 for i in range(N+1)])])
    P = matrix(P_0)
    
    q = matrix(np.append((-1)*mean, [kappa]))
    
    
    A = matrix(np.array([[1.0 for i in range(N)] + [0.0]]), tc='d')
    b = matrix(np.array([1.0]), tc='d')
    
    I = matrix(0.0, (N+1,N+1))
    I[::N+2] = 1.0
    G_1 = np.hstack([(-1)*L.T, np.array([[0.0] for i in range(N)])])
    G_1 = np.vstack([np.array([0.0 for i in range(N)] + [-1.0]), G_1])
    G = matrix([-I, matrix(G_1)])
    h = matrix((N+1)*[0.0] + (N+1)*[0.0])
    
    dims = {'l': N+1, 'q': [N+1], 's': []}
    
    sol = solvers.coneqp(P, q, G, h, dims, A, b)
    w = sol['x'][:-1]
    CE_0 = np.dot(mean,w)[0] - 0.5*a*np.dot(np.dot(w.T, cov), w)[0][0]
    
    return w, CE_0

In [42]:
def Robust_sensitivity_analysis(mean, cov, sigma, t, k, eta, objective):
    N = len(mean)
    a = 2/t
    kappa = np.sqrt(chi2.ppf(eta, df = N))
    L = np.linalg.cholesky(sigma)
    
    if objective == 'mean':
        z = np.random.normal(size = N)*k + 1
        mean_new = np.multiply(mean, z)
        P_0 = np.hstack([a*cov, np.array([[0.0] for i in range(N)])])
        P_0 = np.vstack([P_0, np.array([0.0 for i in range(N+1)])])
        P = matrix(P_0)
    
        q = matrix(np.append((-1)*mean_new, [kappa]))
    
    
    A = matrix(np.array([[1.0 for i in range(N)] + [0.0]]), tc='d')
    b = matrix(np.array([1.0]), tc='d')
    
    I = matrix(0.0, (N+1,N+1))
    I[::N+2] = 1.0
    G_1 = np.hstack([(-1)*L, np.array([[0.0] for i in range(N)])])
    G_1 = np.vstack([np.array([0.0 for i in range(N)] + [-1.0]), G_1])
    G = matrix([-I, matrix(G_1)])
    h = matrix((N+1)*[0.0] + (N+1)*[0.0])
    
    dims = {'l': N+1, 'q': [N+1], 's': []}
    
    sol = solvers.coneqp(P, q, G, h, dims, A, b)
    w = sol['x'][:-1]
    CE_x = np.dot(mean,w)[0] - 0.5*a*np.dot(np.dot(w.T, cov), w)[0][0]
    
    return w, CE_x

In [41]:
w_r, CE_r_0 = Robust_portfolio(sample_mean, cov, sigma, t, eta = 0.95)

In [43]:
print('The effects of errors in mean: ')
for k in k_list:
    CEL_r = 0
    for i in range(iter_num):
        w_r_x, CE_r_x = Robust_sensitivity_analysis(sample_mean, cov, sigma, t, k, eta = 0.95, objective = 'mean')
        CEL_r += (1 - CE_r_x/CE_r_0)
    CEL_r /= iter_num
    print('When k = ',k, 'the average cash equivalent loss is: ', CEL_r)

The effects of errors in mean: 
When k =  0.05 the average cash equivalent loss is:  0.003316281677130194
When k =  0.1 the average cash equivalent loss is:  0.006676292360710063
When k =  0.15000000000000002 the average cash equivalent loss is:  0.016372399018444103
When k =  0.2 the average cash equivalent loss is:  0.01165436180575672


### Questions

1. Not sure whether the sigma should change when covariance matrix changes  
2. Should use sample_cov or true_cov to conduct traditional optimizaiton for result comparision